In [117]:
import hashlib
from random import randrange, choice
import string
import numpy as np
import subprocess
import hashid

In [ ]:
def jumbo_password(password,special = False):
    list_password = list(password)
    list_special = ['a','A','O','t','E','I','S','e']
    lst_index_min = []
    lst_index_maj = []
    lst_index_spec = []
    for i,s in enumerate(list_password):
        if s in string.ascii_lowercase:
            lst_index_min.append(i)
        if s in string.ascii_uppercase:
            lst_index_maj.append(i)
        if s in list_special:
            lst_index_spec.append(i)

    index_min = len(password) + 1
    index_maj = len(password) + 1
    if len(lst_index_min) > 0:
        index_min = choice(lst_index_min)
        list_password[index_min] = list_password[index_min].upper()
    if len(lst_index_maj) > 0:
        index_maj = choice(lst_index_maj)
        list_password[index_maj] = list_password[index_maj].lower()
    if special:
        if len(lst_index_spec) > 0:
            lst_index_spec = [e for e in lst_index_spec if e not in (index_min, index_maj)]
            if len(lst_index_spec) > 0:
                index_spec = choice(lst_index_spec)
                if(list_password[index_spec] == 'a'):
                    list_password[index_spec] = '@'
                if(list_password[index_spec] == 'A'):
                    list_password[index_spec] = '4'
                if(list_password[index_spec] == 'O'):
                    list_password[index_spec] = '0'
                if(list_password[index_spec] == 't'):
                    list_password[index_spec] = '+'
                if(list_password[index_spec] == 'E'):
                    list_password[index_spec] = '3'
                if(list_password[index_spec] == 'I'):
                    list_password[index_spec] = '1'
                if(list_password[index_spec] == 'S'):
                    list_password[index_spec] = '5'
                if(list_password[index_spec] == 'e'):
                    list_password[index_spec] = '€'

    return ''.join(list_password)

In [208]:
def defend_hash(password,strength,outfile = 'password.txt'):
    list_hashing = ['md5','sha256']
    hashing = list_hashing[randrange(len(list_hashing))]
    if strength == 2:
        password = jumbo_password(password)
    if strength == 3:
        password = jumbo_password(password,'True')
    if hashing == 'md5':
        result = hashlib.md5(password.encode())
    if hashing == 'sha256':
        result = hashlib.sha256(password.encode())
    #print(f'Le mot de passe est {password} chiffrée avec {hashing} et pour résultat {result.hexdigest()}')
    with open(outfile, 'w') as f:
        f.write(result.hexdigest())
    return password, result.hexdigest()

In [180]:
defend_hash("princess",2)

Le mot de passe est princEss chiffrée avec md5 et pour résultat f7d9a06033b8da9ce7f33ce8c9513f59


('princEss', 'f7d9a06033b8da9ce7f33ce8c9513f59')

In [203]:
def attack_hash(strength,wordlist='short_ech.txt',file = 'password.txt'):
    guess = ''
    list_hashing = ['raw-md5','raw-sha256']
    if strength == 1:
        hash = choice(list_hashing)
        subprocess.run(['john','--wordlist='+wordlist,'--format='+hash, file], stdout=subprocess.PIPE).stdout.decode('utf-8')
        res = subprocess.run(['john', '--show','--format='+hash, file], stdout=subprocess.PIPE).stdout.decode('utf-8')
    if strength == 2:
        with open(file) as f:
            lines = f.readlines()
            for mode in hashID.identifyHash(lines[0]):
                if mode.john in list_hashing:
                    hash = mode.john
        subprocess.run(['john','--wordlist='+wordlist,'--format='+hash, file], stdout=subprocess.PIPE).stdout.decode('utf-8')
        res = subprocess.run(['john', '--show','--format='+hash, file], stdout=subprocess.PIPE).stdout.decode('utf-8')
    if strength == 3:
        with open(file) as f:
            lines = f.readlines()
            for mode in hashID.identifyHash(lines[0]):
                if mode.john in list_hashing:
                    hash = mode.john
        subprocess.run(['john','--rules=Jumbo','--wordlist='+wordlist,'--format='+hash, file], stdout=subprocess.PIPE).stdout.decode('utf-8')
        res = subprocess.run(['john', '--show','--format='+hash, file], stdout=subprocess.PIPE).stdout.decode('utf-8')
    if '0 password hashes cracked' not in res:
        guess = res.split(':')[1].split('\n')[0]
    return guess


In [209]:
with open('short_ech.txt') as f:
    alist = [line.rstrip() for line in f]

count = 0
for i in range(100):
    password = choice(alist)
    answer, _ = defend_hash(password, 2)
    guess = attack_hash(3)
    if answer == guess:
        count += 1

print(f'Le pourcentage de réussite est de {count}%')

Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Using default input encoding: UTF-8
Will run 2 OpenMP threads
Press Ctrl-C to abort, or send SIGUSR1 to john process for status
1g 0:00:00:00 DONE (2022-01-11 09:24) 4.761g/s 84580p/s 84580c/s 84580C/s password..iloveÿou
Use the "--show --format=Raw-SHA256" options to display all of the cracked passwords reliably
Session completed. 
Using default input encoding: UTF-8
Press Ctrl-C to abort, or send SIGUSR1 to john process for status
1g 0:00:00:00 DONE (202

Le pourcentage de réussite est de 100%


Using default input encoding: UTF-8
